#### 라이브러리 호출

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

import matplotlib.font_manager as fm

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

In [24]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

In [25]:
# 라이브러리 호출
import pandas as pd
from sklearn.model_selection import train_test_split

# 경고창 제거
import warnings
warnings.filterwarnings('ignore')

In [26]:
plt.rc('font', family='Malgun Gothic') # 폰트 지정
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
%config InlineBackend.figure_format='retina' # 그래프 글씨 뚜렷

#### 데이터 셋 불러오기

In [27]:
# train data : 4개월 데이터
train_df = pd.read_csv('../model_dataset/quarter_features.csv',index_col=0)
train_df.head()
# test data : 19일 데이터 
test_df = pd.read_csv("../model_dataset/test_features.csv",index_col=0)
test_df.head()

,시장베이시스,이론베이시스,괴리율,원위안 야간변동율,원엔 야간변동율,KOSPI 전날변동율,KOSPI 야간변동율,KS200 전날변동율,KS200 야간변동율,NAS 야간변동율,...,원달러 야간변동율,VIX 전날변동율,VIX 당일변동율,VIX 당일변화량,JNIV 종가변동율,JNIV 전날변동율,JNIV 전날변화량,CD 전날변동율,CD 전날변화량,VKOSPI_Label
2023-10-10,1.56,2.31,-0.23,0.070,0.056,0.21,1.156,0.11,1.264,0.39,...,0.086,1.43,-9.417,2.04,-8.44,1.214,0.76,-0.261,0.01,1
2023-10-11,2.34,2.17,0.05,0.109,0.105,-0.26,1.413,0.05,1.520,0.58,...,0.096,-3.79,-3.785,1.35,-1.78,-1.964,0.92,0.000,0.00,0
2023-10-12,2.26,2.18,0.02,0.093,0.105,1.98,0.617,1.76,0.662,0.71,...,0.084,-5.52,-5.074,1.69,3.03,1.983,0.69,0.000,0.00,0
2023-10-13,1.60,2.17,-0.17,0.043,0.036,1.21,-0.765,1.27,-0.833,-0.63,...,0.033,3.73,3.794,2.64,-1.99,1.423,0.86,0.000,0.00,0
2023-10-16,2.08,2.11,-0.01,0.081,0.124,-0.95,-0.559,-0.94,-0.513,-1.23,...,0.064,15.76,16.878,4.28,0.05,0.878,0.93,0.000,0.00,0


In [28]:
# train/test셋 분리
X_train = train_df.drop(columns='VKOSPI_Label',axis=1)
y_train = train_df[['VKOSPI_Label']]
X_val = test_df.drop(columns='VKOSPI_Label',axis=1)
y_val = test_df[['VKOSPI_Label']]

### Scaling 진행
* Standard
* Minmax 

In [29]:
# 표준화
features_to_standardize = ['시장베이시스', '이론베이시스', '괴리율', '원위안 야간변동율', '원엔 야간변동율', 'KOSPI 전날변동율',
       'KOSPI 야간변동율', 'KS200 전날변동율', 'KS200 야간변동율', 'NAS 야간변동율', 'NAS 당일변동율',
       'NAS 당일변화량', 'P/C Ratio', 'CALL_vol_change(%)', 'PUT_vol_change(%)',
       'CALL_vol_fluc', 'PUT_vol_fluc', 'S&P 야간변동율', 'S&P 당일변동율', 'S&P 당일변화량',
       '원달러 야간변동율', 'VIX 전날변동율', 'VIX 당일변동율', 'VIX 당일변화량', 'JNIV 종가변동율',
       'JNIV 전날변동율', 'JNIV 전날변화량', 'CD 전날변동율', 'CD 전날변화량']
scaler_standardize = StandardScaler()
X_train = scaler_standardize.fit_transform(X_train[features_to_standardize])
X_val = scaler_standardize.transform(X_val[features_to_standardize])

X_train = pd.DataFrame(X_train, columns=features_to_standardize)
X_val = pd.DataFrame(X_val, columns=features_to_standardize)

In [30]:
# minmax 스케일링

# features_to_minmax = ['시장베이시스', '이론베이시스', '괴리율', '원위안 야간변동율', '원엔 야간변동율', 'KOSPI 전날변동율',
#        'KOSPI 야간변동율', 'KS200 전날변동율', 'KS200 야간변동율', 'NAS 야간변동율', 'NAS 당일변동율',
#        'NAS 당일변화량', 'P/C Ratio', 'CALL_vol_change(%)', 'PUT_vol_change(%)',
#        'CALL_vol_fluc', 'PUT_vol_fluc', 'S&P 야간변동율', 'S&P 당일변동율', 'S&P 당일변화량',
#        '원달러 야간변동율', 'VIX 전날변동율', 'VIX 당일변동율', 'VIX 당일변화량', 'JNIV 종가변동율',
#        'JNIV 전날변동율', 'JNIV 전날변화량', 'CD 전날변동율', 'CD 전날변화량']

# scaler_minmax = MinMaxScaler()
# X_train = scaler_minmax.fit_transform(X_train[features_to_minmax])
# X_val = scaler_minmax.transform(X_val[features_to_minmax])

# X_train = pd.DataFrame(X_train, columns=features_to_minmax)
# X_val = pd.DataFrame(X_val, columns=features_to_minmax)

### RF 기반 feature importance

In [31]:
# # Feature_Importances

# rfc = RandomForestClassifier(random_state=42)

# rfc.fit(X_train, y_train)

# rfc.feature_importances_

# ftr_importances = pd.Series(rfc.feature_importances_, index=X_train.columns)
# sorted_feature_importance = ftr_importances.sort_values(ascending=True)

# data = sorted_feature_importance
# importance = pd.DataFrame(data, columns=['feature importances'])
# importance

In [32]:
# plt.figure(figsize=(10, 6))
# colors = plt.cm.tab20(np.linspace(0, 1, len(importance)))

# plt.barh(importance.index, importance['feature importances'], color=colors)  # Changed to barh for horizontal bar chart
# plt.ylabel('Features')  # Changed to 'Features' for Y-axis label
# plt.xlabel('Importance')  # Changed to 'Importance' for X-axis label
# plt.title('Feature Importances')

# # Rotating x labels for better visibility if needed
# plt.xticks(rotation=45)

# # Show the plot
# plt.tight_layout()
# plt.show()

### Logit기반 피쳐 갯수 선정

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
import statsmodels.api as sm
import numpy as np
lr_clf = LogisticRegression()


logit = SelectFromModel(LogisticRegression())
logit.fit(X_train, y_train)
logit_support = logit.get_support()
lr_feature = X_train.loc[:,logit_support].columns.tolist()

In [34]:
print(lr_feature)
print(len(lr_feature))

['시장베이시스', '원엔 야간변동율', 'KOSPI 전날변동율', 'KS200 야간변동율', 'NAS 야간변동율', 'NAS 당일변화량', 'CALL_vol_change(%)', 'PUT_vol_change(%)', 'CALL_vol_fluc', 'PUT_vol_fluc', 'S&P 당일변화량', 'VIX 당일변화량', 'JNIV 종가변동율']
13


In [35]:
lr_feature=['시장베이시스', '원엔 야간변동율', 'KOSPI 전날변동율', 'KS200 야간변동율', 'NAS 야간변동율', 'NAS 당일변화량', 'CALL_vol_change(%)', 
            'PUT_vol_change(%)', 'CALL_vol_fluc', 'PUT_vol_fluc', 'S&P 당일변화량', 'VIX 당일변화량', 'JNIV 종가변동율']

### 1. Wrapper : 모델링을 돌리면서 변수를 선택하는 방법

- Forward Selection(전진 선택) : 변수가 없는 상태로 시작하며 반복할 때마다 가장 중요한 변수를 추가하여 더 이상 성능의 향상이 없을 때까지 변수를 추가한다.
- Backward Elimination(후방 제거) : 모든 변수를 가지고 시작하며, 가장 덜 중요한 변수를 하나씩 제거하면서 모델의 성능을 향상시킨다. 더 이상 성능의 향상이 없을 때까지 반복한다.
- Stepwise Selection(단계별 선택): Foward Selection 과 Backward Elimination 을 결합하여 사용하는 방식으로, 모든  변수를 가지고 시작하여 가장 도움이 되지 않는 변수를 삭제하거나, 모델에서 빠져있는 변수 중에서 가장 중요한 변수를 추가하는 방법이다. 이와 같이 변수를 추가 또는 삭제를 반복한다. 반대로 아무것도 없는 모델에서 출발해 변수를 추가, 삭제를 반복할 수도 있다.

In [36]:
# # Forward Selection
# import pandas as pd
# from sklearn.linear_model import LogisticRegression

# # Forward feature selection 수행
# selected_features = []
# best_score = 0

# while len(selected_features) < train_df.shape[1]:
#     best_feature = None
#     best_model = None
#     best_score_local = 0

#     for feature in X_train.columns:
#         if feature not in selected_features:
#             features = selected_features + [feature]
#             X_train_selected = X_train[features]
#             X_val_selected = X_val[features]

#             model = LogisticRegression()
#             model.fit(X_train_selected, y_train)
#             score = model.score(X_val_selected, y_val)

#             if score > best_score_local:
#                 best_score_local = score
#                 best_feature = feature
#                 best_model = model

#     if best_score_local > best_score:
#         selected_features.append(best_feature)
#         best_score = best_score_local
#         print(f"Selected feature: {best_feature}, Accuracy: {best_score:.4f}")

#     else:
#         break

# print("\nForward selected features:")
# Forward = selected_features


In [37]:
# # Backward Elimination
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression

# # Backward feature selection 수행
# selected_features = X_train.columns.tolist()
# best_score = 0

# while len(selected_features) > 0:
#     worst_feature = None
#     best_model = None
#     best_score_local = 0

#     for feature in selected_features:
#         features = selected_features.copy()
#         features.remove(feature)

#         X_train_selected = X_train[features]
#         X_val_selected = X_val[features]

#         model = LogisticRegression()
#         model.fit(X_train_selected, y_train)
#         score = model.score(X_val_selected, y_val)

#         if score > best_score_local:
#             best_score_local = score
#             worst_feature = feature
#             best_model = model

#     if best_score_local > best_score:
#         selected_features.remove(worst_feature)
#         best_score = best_score_local
#         print(f"Removed feature: {worst_feature}, Accuracy: {best_score:.4f}")

#     else:
#         break

# print("\nFinal selected features:")
# Backward = selected_features


In [38]:
# # Stepwise Selection
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression

# # Stepwise feature selection 수행
# selected_features = []
# best_score = 0

# # Forward step
# while len(selected_features) < train_df.shape[1]:
#     best_feature = None
#     best_model = None
#     best_score_local = 0

#     for feature in X_train.columns:
#         if feature not in selected_features:
#             features = selected_features + [feature]
#             X_train_selected = X_train[features]
#             X_val_selected = X_val[features]

#             model = LogisticRegression()
#             model.fit(X_train_selected, y_train)
#             score = model.score(X_val_selected, y_val)

#             if score > best_score_local:
#                 best_score_local = score
#                 best_feature = feature
#                 best_model = model

#     if best_score_local > best_score:
#         selected_features.append(best_feature)
#         best_score = best_score_local
#         print(f"Selected feature: {best_feature}, Accuracy: {best_score:.4f}")

#     else:
#         break

# # Backward step
# while len(selected_features) > 0:
#     worst_feature = None
#     best_model = None
#     best_score_local = 0

#     for feature in selected_features:
#         features = selected_features.copy()
#         features.remove(feature)

#         X_train_selected = X_train[features]
#         X_val_selected = X_val[features]

#         model = LogisticRegression()
#         model.fit(X_train_selected, y_train)
#         score = model.score(X_val_selected, y_val)

#         if score > best_score_local:
#             best_score_local = score
#             worst_feature = feature
#             best_model = model

#     if best_score_local > best_score:
#         selected_features.remove(worst_feature)
#         best_score = best_score_local
#         print(f"Removed feature: {worst_feature}, Accuracy: {best_score:.4f}")

#     else:
#         break

# print("\nFinal selected features:")
# Stepwise = selected_features


## 3. Embedded
- Lasso, Ridge, Elastic Net 등 내장함수 사용하여 변수를 선택하는 방법

In [39]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.simplefilter('ignore')

lasso_model = LogisticRegression()
param_grid = {'penalty' : ['l1'], 
                'C' : [0.001, 0.01, 0.1, 1, 2, 5, 10],
                'solver' : ['liblinear']}

grid_search = GridSearchCV(lasso_model, param_grid=param_grid, return_train_score=True, cv=5)
grid_search.fit(X_train, y_train)

df = pd.DataFrame(grid_search.cv_results_)
df = df.sort_values(by=['rank_test_score'], ascending=True)
df[['params', 'mean_train_score', 'mean_test_score', 'rank_test_score']]
print('GridSearchCV 최적 파라미터:', grid_search.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f}'.format(grid_search.best_score_))

GridSearchCV 최적 파라미터: {'C': 0.001, 'penalty': 'l1', 'solver': 'liblinear'}
GridSearchCV 최고 정확도:0.7750


In [40]:
lasso_best = LogisticRegression(C=1, penalty='l1', solver='liblinear').fit(X_train, y_train)

df_lasso = pd.DataFrame()
df_lasso['feature'] = X_train.columns
df_lasso['coef'] = lasso_best.coef_[0]
df_lasso.drop(df_lasso[df_lasso['coef']==0].index, inplace=True)
df_lasso

,feature,coef
1,이론베이시스,0.198905
4,원엔 야간변동율,-0.254335
5,KOSPI 전날변동율,-0.196586
8,KS200 야간변동율,-0.147170
15,CALL_vol_fluc,0.320481
16,PUT_vol_fluc,0.074957
19,S&P 당일변화량,-0.294597
21,VIX 전날변동율,0.133319
23,VIX 당일변화량,0.143465
24,JNIV 종가변동율,0.427095


In [41]:
# 라쏘에서 선택된 피처
lasso = df_lasso['feature'].values.tolist()
print('Lasso에서 선택된 피처 수 {0:1.0f}'.format(len(df_lasso)), '개')
lasso

Lasso에서 선택된 피처 수 12 개


['이론베이시스',
 '원엔 야간변동율',
 'KOSPI 전날변동율',
 'KS200 야간변동율',
 'CALL_vol_fluc',
 'PUT_vol_fluc',
 'S&P 당일변화량',
 'VIX 전날변동율',
 'VIX 당일변화량',
 'JNIV 종가변동율',
 'JNIV 전날변화량',
 'CD 전날변동율']

#### 로지 라쏘 피처 셀렉

In [42]:
list_logistic = list(lr_feature)
list_lasso = list(lasso)
list_col_all = X_train.columns

In [43]:
def func_logistic(x):
    if x in list_logistic:
        return 1
    else:
        return 0

    
def func_lasso(x):
    if x in list_lasso:
        return 1
    else:
        return 0

In [44]:
# 2번 선택된 Feature

feature_counts = pd.DataFrame()
feature_counts['Feature'] = list_col_all
feature_counts['logistic'] = list_col_all.map(func_logistic)
feature_counts['lasso'] = list_col_all.map(func_lasso)

feature_counts["total"] = feature_counts["logistic"]+feature_counts['lasso']
feature_final = feature_counts[feature_counts["total"]>=2]
list_feature_final = list(feature_final["Feature"])
print("선택된 피쳐수 :", len(list_feature_final))
feature_final['Feature'].to_list()

선택된 피쳐수 : 8


['원엔 야간변동율',
 'KOSPI 전날변동율',
 'KS200 야간변동율',
 'CALL_vol_fluc',
 'PUT_vol_fluc',
 'S&P 당일변화량',
 'VIX 당일변화량',
 'JNIV 종가변동율']

In [45]:
selected_feature = ['원엔 야간변동율',
 'KOSPI 전날변동율',
 'KS200 야간변동율',
 'CALL_vol_fluc',
 'PUT_vol_fluc',
 'S&P 당일변화량',
 'VIX 당일변화량',
 'JNIV 종가변동율']

#### VIF 확인

In [46]:
def vif(data):
    import pandas as pd
    from statsmodels.stats.outliers_influence import variance_inflation_factor

    # VIF 출력을 위한 데이터 프레임 형성
    vif = pd.DataFrame()

    # VIF 값과 각 Feature 이름에 대해 설정
    vif["VIF Factor"] = [variance_inflation_factor(data.values, i) for i in range(len(data.columns))]
    vif["features"] = data.columns 
                    
    # VIF 값이 높은 순으로 정렬
    vif = vif.sort_values(by="VIF Factor", ascending=False)
    vif = vif.reset_index().drop(columns='index')
    
    return vif

vif(X_train[selected_feature])

,VIF Factor,features
0,6.632427,CALL_vol_fluc
1,6.371169,PUT_vol_fluc
2,1.919559,VIX 당일변화량
3,1.487224,S&P 당일변화량
4,1.373893,KOSPI 전날변동율
5,1.132701,KS200 야간변동율
6,1.125545,JNIV 종가변동율
7,1.016476,원엔 야간변동율


#### 이전코드

In [48]:
# list_Forward = list(Forward)
# list_Backward = list(Backward)
# list_Stepwise = list(Stepwise)
# list_lasso = list(lasso)
# list_col_all = X_train.columns

In [49]:
# def func_Forward(x):
#     if x in list_Forward:
#         return 1
#     else:
#         return 0
    
# def func_Backward(x):
#     if x in list_Backward:
#         return 1
#     else:
#         return 0


# def func_Stepwise(x):
#     if x in list_Stepwise:
#         return 1
#     else:
#         return 0

    
# def func_lasso(x):
#     if x in list_lasso:
#         return 1
#     else:
#         return 0

In [50]:
# # 2번 선택된 Feature

# feature_counts = pd.DataFrame()
# feature_counts['Feature'] = list_col_all
# feature_counts['Forward'] = list_col_all.map(func_Forward)
# feature_counts['Backward'] = list_col_all.map(func_Backward)
# feature_counts['Stepwise'] = list_col_all.map(func_Stepwise)
# feature_counts['lasso'] = list_col_all.map(func_lasso)

# feature_counts["total"] = feature_counts["Forward"]+feature_counts['Backward']+feature_counts['Stepwise']+feature_counts["lasso"]
# feature_final = feature_counts[feature_counts["total"]>=2]
# list_feature_final = list(feature_final["Feature"])
# print("선택된 피쳐수 :", len(list_feature_final))
# feature_final

In [51]:
# np.array(feature_final[['Feature']])


In [52]:
# # 3번 선택된 Feature

# feature_counts = pd.DataFrame()
# feature_counts['Feature'] = list_col_all
# feature_counts['Forward'] = list_col_all.map(func_Forward)
# feature_counts['Backward'] = list_col_all.map(func_Backward)
# feature_counts['Stepwise'] = list_col_all.map(func_Stepwise)
# feature_counts['lasso'] = list_col_all.map(func_lasso)

# feature_counts["total"] = feature_counts["Forward"]+feature_counts['Backward']+feature_counts['Stepwise']+feature_counts["lasso"]
# feature_final = feature_counts[feature_counts["total"]>=3]
# list_feature_final = list(feature_final["Feature"])
# print("선택된 피쳐수 :", len(list_feature_final))
# feature_final

## 모델링

In [53]:
# X_train = X_train[['이론베이시스',
# '괴리율',
# '원위안 야간변동율',
# 'KS200 야간변동율',
# 'PUT_vol_change(%)',
# 'CALL_vol_fluc',
# 'S&P 당일변화량',
# 'VIX 전날변동율',
# 'VIX 당일변화량',
# 'JNIV 종가변동율']]
# X_val = X_val[['이론베이시스',
# '괴리율',
# '원위안 야간변동율',
# 'KS200 야간변동율',
# 'PUT_vol_change(%)',
# 'CALL_vol_fluc',
# 'S&P 당일변화량',
# 'VIX 전날변동율',
# 'VIX 당일변화량',
# 'JNIV 종가변동율']]

In [54]:
# 피처 셀렉 이후 모델링
X_train = X_train[['원엔 야간변동율',
 'KOSPI 전날변동율',
 'KS200 야간변동율',
 'CALL_vol_fluc',
 'PUT_vol_fluc',
 'S&P 당일변화량',
 'VIX 당일변화량',
 'JNIV 종가변동율']]
X_val = X_val[['원엔 야간변동율',
 'KOSPI 전날변동율',
 'KS200 야간변동율',
 'CALL_vol_fluc',
 'PUT_vol_fluc',
 'S&P 당일변화량',
 'VIX 당일변화량',
 'JNIV 종가변동율']]

### ADASYN

In [55]:
from imblearn.over_sampling import ADASYN

ada = ADASYN(sampling_strategy='minority', random_state=42)
X_train, y_train = ada.fit_resample(X_train, y_train)

In [56]:
#### 오버샘플링 적용 (adasyn)
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
def model_basic(X_train, y_train, X_test, y_test):
    models = [
        LogisticRegression(),
        DecisionTreeClassifier(),
        SVC(),
        RandomForestClassifier(),
        XGBClassifier(),
        LGBMClassifier()
    ]

    rdict={'model':[],'accuracy':[],'precision':[],'recall':[],'f1_score':[]}

    
    for clf in models:
        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        results = (round(accuracy_score(y_test, pred),2),
                    round(precision_score(y_test, pred),2),
                    round(recall_score(y_test, pred),2),
                    round(f1_score(y_test, pred),2))
        rdict['model'].append(clf); 
        rdict['accuracy'].append(results[0])
        rdict['precision'].append(results[1])
        rdict['recall'].append(results[2])
        rdict['f1_score'].append(results[3])

        # print(results)

    rdf = pd.DataFrame(data=rdict)
    return rdf 

In [57]:
model_basic(X_train, y_train, X_val, y_val)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 65, number of negative: 62
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000136 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 346
[LightGBM] [Info] Number of data points in the train set: 127, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.511811 -> initscore=0.047253
[LightGBM] [Info] Start training from score 0.047253
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

,model,accuracy,precision,recall,f1_score
0,LogisticRegression(),0.63,0.33,0.4,0.36
1,DecisionTreeClassifier(),0.53,0.00,0.0,0.00
2,SVC(),0.53,0.00,0.0,0.00
3,"(DecisionTreeClassifier(max_features='sqrt', r...",0.47,0.00,0.0,0.00
4,"XGBClassifier(base_score=None, booster=None, c...",0.58,0.20,0.2,0.20
5,LGBMClassifier(),0.63,0.25,0.2,0.22


#### 하이퍼파라미터 조정

In [58]:
# from sklearn.model_selection import GridSearchCV

# def model_basic_with_gridsearch(X_train, y_train, X_val, y_val):
#     # 모델 및 해당 하이퍼파라미터 그리드 정의
#     models_and_grids = {
#         LogisticRegression(): {
#             'penalty': ['l1', 'l2'],
#             'C': [0.01, 0.1, 1, 10]
#         },
#         DecisionTreeClassifier(): {
#             'max_depth': [1,3,7,10],
#             'min_samples_split': [2, 5, 10],
#             'min_samples_leaf': [1, 2, 4]
#         },
#         SVC(): {
#             'C': [0.1, 1, 10],
#             'kernel': ['linear', 'rbf']
#         },
#         RandomForestClassifier(): {
#             'n_estimators': [5,10,15,20],
#             'max_depth': [1,3,7,10],
#             'min_samples_split': [2, 5, 10]
#         },
#         XGBClassifier(): {
#             'n_estimators': [5,10,15,20],
#             'learning_rate': [0.01, 0.1, 0.5]
#         },
#         LGBMClassifier(): {
#             'n_estimators': [5,10,15,20],
#             'learning_rate': [0.01, 0.1, 0.5]
#         }
#     }

#     rdict = {'model': [], 'best_params': [], 'accuracy': [], 'precision': [], 'recall': [], 'f1_score': []}

#     for model, grid in models_and_grids.items():
#         clf = GridSearchCV(model, grid, cv=5)
#         clf.fit(X_train, y_train)
#         best_model = clf.best_estimator_

#         pred = best_model.predict(X_val)
#         results = (round(accuracy_score(y_val, pred), 2),
#                    round(precision_score(y_val, pred, average='weighted'), 2),
#                    round(recall_score(y_val, pred, average='weighted'), 2),
#                    round(f1_score(y_val, pred, average='weighted'), 2))
#         print(clf.best_params_)
#         rdict['model'].append(best_model.__class__.__name__)
#         rdict['best_params'].append(clf.best_params_)
#         rdict['accuracy'].append(results[0])
#         rdict['precision'].append(results[1])
#         rdict['recall'].append(results[2])
#         rdict['f1_score'].append(results[3])
    
#     rdf = pd.DataFrame(data=rdict)
#     return rdf


In [59]:
# model_basic_with_gridsearch(X_train, y_train, X_val, y_val)

#### 모델 성능 평가

In [60]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score


import warnings

def get_clf_eval(MDA_y_test, pred):
    confusion = confusion_matrix(MDA_y_test, pred)
    accuracy = accuracy_score(MDA_y_test, pred)
    precision = precision_score(MDA_y_test, pred)
    recall = recall_score(MDA_y_test, pred)
    roc_score = roc_auc_score(MDA_y_test, pred)
    pr_score = average_precision_score(MDA_y_test, pred)
    f1 = f1_score(MDA_y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))

In [61]:
# 임계값에 따른 오차행렬및 스코어 -------->#임계값 최적 : 재현율기준(0.1)/f1기준(0.3)
from sklearn.preprocessing import Binarizer
thresholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]

def get_eval_by_threshold(MDA_y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print("임곗값:", custom_threshold)
        print("pred:", custom_predict)
        get_clf_eval(MDA_y_test, custom_predict)
        print("---------------------------------------------------------")

In [62]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
lda_pred = dt.predict(X_val)
lda_pred_proba = dt.predict_proba(X_val)

get_clf_eval(y_val, lda_pred)
get_eval_by_threshold(y_val , lda_pred_proba[:,1].reshape(-1,1), thresholds)

오차행렬
[[10  4]
 [ 5  0]]
정확도: 0.5263, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.3571, PR score : 0.2632
임곗값: 0.1
pred: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[10  4]
 [ 5  0]]
정확도: 0.5263, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.3571, PR score : 0.2632
---------------------------------------------------------
임곗값: 0.2
pred: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[10  4]
 [ 5  0]]
정확도: 0.5263, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.3571, PR score : 0.2632
---------------------------------------------------------
임곗값: 0.3
pred: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[10  4]
 [ 5  0]]
정확도: 0.5263, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.3571, PR score : 0.2632
--------------------------------------------------

In [63]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
lda_pred = rf.predict(X_val)
lda_pred_proba = rf.predict_proba(X_val)

get_clf_eval(y_val, lda_pred)
get_eval_by_threshold(y_val , lda_pred_proba[:,1].reshape(-1,1), thresholds)

오차행렬
[[9 5]
 [5 0]]
정확도: 0.4737, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.3214, PR score : 0.2632
임곗값: 0.1
pred: [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
오차행렬
[[ 0 14]
 [ 0  5]]
정확도: 0.2632, 정밀도 : 0.2632, 재현율:1.0000,F1 스코어:0.4167
ROC 스코어: 0.5000, PR score : 0.2632
---------------------------------------------------------
임곗값: 0.2
pred: [[0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]]
오차행렬
[[ 2 12]
 [ 2  3]]
정확도: 0.2632, 정밀도 : 0.2000, 재현율:0.6000,F1 스코어:0.3000
ROC 스코어: 0.3714, PR score : 0.2253
---------------------------------------------------------
임곗값: 0.3
pred: [[0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[6 8]
 [3 2]]
정확도: 0.4211, 정밀도 : 0.2000, 재현율:0.4000,F1 스코어:0.2667
ROC 스코어: 0.4143, PR score : 0.2379
---------------------------------------------------------


In [64]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
lda_pred = xgb.predict(X_val)
lda_pred_proba = xgb.predict_proba(X_val)

get_clf_eval(y_val, lda_pred)
get_eval_by_threshold(y_val , lda_pred_proba[:,1].reshape(-1,1), thresholds)

오차행렬
[[10  4]
 [ 4  1]]
정확도: 0.5789, 정밀도 : 0.2000, 재현율:0.2000,F1 스코어:0.2000
ROC 스코어: 0.4571, PR score : 0.2505
임곗값: 0.1
pred: [[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[7 7]
 [1 4]]
정확도: 0.5789, 정밀도 : 0.3636, 재현율:0.8000,F1 스코어:0.5000
ROC 스코어: 0.6500, PR score : 0.3435
---------------------------------------------------------
임곗값: 0.2
pred: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[7 7]
 [4 1]]
정확도: 0.4211, 정밀도 : 0.1250, 재현율:0.2000,F1 스코어:0.1538
ROC 스코어: 0.3500, PR score : 0.2355
---------------------------------------------------------
임곗값: 0.3
pred: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[8 6]
 [4 1]]
정확도: 0.4737, 정밀도 : 0.1429, 재현율:0.2000,F1 스코어:0.1667
ROC 스코어: 0.3857, PR score : 0.2391
---------------------------------------------------------
임곗값:

In [65]:
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)
lda_pred = lgbm.predict(X_val)
lda_pred_proba = lgbm.predict_proba(X_val)

get_clf_eval(y_val, lda_pred)
get_eval_by_threshold(y_val , lda_pred_proba[:,1].reshape(-1,1), thresholds)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 65, number of negative: 62
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000125 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 346
[LightGBM] [Info] Number of data points in the train set: 127, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.511811 -> initscore=0.047253
[LightGBM] [Info] Start training from score 0.047253
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

오차행렬
[[8 6]
 [1 4]]
정확도: 0.6316, 정밀도 : 0.4000, 재현율:0.8000,F1 스코어:0.5333
ROC 스코어: 0.6857, PR score : 0.3726
---------------------------------------------------------
임곗값: 0.3
pred: [[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[9 5]
 [2 3]]
정확도: 0.6316, 정밀도 : 0.3750, 재현율:0.6000,F1 스코어:0.4615
ROC 스코어: 0.6214, PR score : 0.3303
---------------------------------------------------------
임곗값: 0.4
pred: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[10  4]
 [ 4  1]]
정확도: 0.5789, 정밀도 : 0.2000, 재현율:0.2000,F1 스코어:0.2000
ROC 스코어: 0.4571, PR score : 0.2505
---------------------------------------------------------
임곗값: 0.5
pred: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[11  3]
 [ 4  1]]
정확도: 0.6316, 정밀도 : 0.2500, 재현율:0.2000,F1 스코어:0.2222
ROC 스코어: 0.4929, PR score : 0.2605


In [66]:
# SVC 모델을 생성
svc = SVC(probability=True)  # probability=True로 설정하여 예측 확률을 얻을 수 있도록 함
svc.fit(X_train, y_train)

# 예측 및 예측 확률 계산
svc_pred = svc.predict(X_val)
svc_pred_proba = svc.predict_proba(X_val)

# 정확도 평가
get_clf_eval(y_val, svc_pred)
get_eval_by_threshold(y_val, lda_pred_proba[:,1].reshape(-1,1),thresholds)

오차행렬


[[10  4]
 [ 5  0]]
정확도: 0.5263, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.3571, PR score : 0.2632
임곗값: 0.1
pred: [[1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[7 7]
 [1 4]]
정확도: 0.5789, 정밀도 : 0.3636, 재현율:0.8000,F1 스코어:0.5000
ROC 스코어: 0.6500, PR score : 0.3435
---------------------------------------------------------
임곗값: 0.2
pred: [[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[8 6]
 [1 4]]
정확도: 0.6316, 정밀도 : 0.4000, 재현율:0.8000,F1 스코어:0.5333
ROC 스코어: 0.6857, PR score : 0.3726
---------------------------------------------------------
임곗값: 0.3
pred: [[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[9 5]
 [2 3]]
정확도: 0.6316, 정밀도 : 0.3750, 재현율:0.6000,F1 스코어:0.4615
ROC 스코어: 0.6214, PR score : 0.3303
---------------------------------------------------------
임곗값: 0.4

In [67]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lda_pred = lr.predict(X_val)
lda_pred_proba = lr.predict_proba(X_val)

get_clf_eval(y_val, lda_pred)
get_eval_by_threshold(y_val , lda_pred_proba[:,1].reshape(-1,1), thresholds)

오차행렬
[[10  4]
 [ 3  2]]
정확도: 0.6316, 정밀도 : 0.3333, 재현율:0.4000,F1 스코어:0.3636
ROC 스코어: 0.5571, PR score : 0.2912
임곗값: 0.1
pred: [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[ 2 12]
 [ 1  4]]
정확도: 0.3158, 정밀도 : 0.2500, 재현율:0.8000,F1 스코어:0.3810
ROC 스코어: 0.4714, PR score : 0.2526
---------------------------------------------------------
임곗값: 0.2
pred: [[0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[ 4 10]
 [ 2  3]]
정확도: 0.3684, 정밀도 : 0.2308, 재현율:0.6000,F1 스코어:0.3333
ROC 스코어: 0.4429, PR score : 0.2437
---------------------------------------------------------
임곗값: 0.3
pred: [[0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]]
오차행렬
[[5 9]
 [2 3]]
정확도: 0.4211, 정밀도 : 0.2500, 재현율:0.6000,F1 스코어:0.3529
ROC 스코어: 0.4786, PR score : 0.2553
------------------------------------------------------

#### 모델링 분석 코드

In [68]:
# y_val.reset_index(drop=True, inplace=True)
# y_train.reset_index(drop=True, inplace=True)

In [69]:
# lda_pred

In [70]:
# a = pd.concat([X_val,y_val],axis=1)
# a["pre"] = lda_pred
# a

In [71]:
# b = pd.concat([X_train,y_train],axis=1)
# b

In [72]:
# b[b["VKOSPI_Label"]==0].describe()

In [73]:
# a[a["VKOSPI_Label"]==0].describe()

In [74]:
# b[b["VKOSPI_Label"]==1].describe()

In [75]:
# a[a["VKOSPI_Label"]==1].describe()

In [76]:
# a[a["VKOSPI_Label"]==0].describe()